In [1]:
import dxdata
import pandas as pd
import os

# Initialize dxdata engine
engine = dxdata.connect(dialect="hive+pyspark")

In [2]:
project = os.popen("dx env | grep project- | awk -F '\t' '{print $2}'").read().rstrip()
record = os.popen("dx describe *dataset | grep  record- | awk -F ' ' '{print $2}'").read().rstrip().split('\n')[0]
DATASET_ID = project + ":" + record
dataset = dxdata.load_dataset(id=DATASET_ID)

In [3]:
data = dataset['participant']


# Find by field name
field_eid = pheno.find_field(name="eid")

# Find by exact title
field_sex = pheno.find_field(title="Sex")
field_age = pheno.find_field(title="Age at recruitment")
field_height = pheno.find_field(title="Standing height | Instance 0")

In [4]:
field_list = [field_eid, field_height, field_sex, field_age]
field_list

[<Field "eid">, <Field "p50_i0">, <Field "p31">, <Field "p21022">]

## prepare recorders 

In [16]:
df_all_fields = pd.DataFrame(columns = ['entity','field_name','filed_title'])
for entity in dataset.entities:
    entity_name  = entity.name
    
    for field in entity.fields:
        field_name = field.name
        filed_title = field.title
        df_all_fields.loc[len(df_all_fields),] = [entity_name,field_name,filed_title]
df_all_fields['downloaded']=[False]*len(df_all_fields)
df_all_fields.to_csv('recorder_all_fields.csv',index=False)


In [24]:
for entity in df_all_fields['entity'].unique():
    df_target_entity = df_all_fields.loc[df_all_fields['entity']==entity,]
    df_target_entity.to_csv(f'recorder_{entity}.csv')

# data downloading codes 

In [ ]:
# 52,68 is missing 

In [22]:
entity='participant'
ind = 18
interval = 21
bulk_num = 30
fail_list = [17,18, 52, 68, 179, 815, 816, 820, 828]

In [23]:
from datetime import datetime
start_time = datetime.now()
df_target_entity = pd.read_csv(f'recorder/recorder_{entity}.csv')
end = ind+interval
ind
while ind < end:
    try:
        print(f"now download bulk {ind}, from {ind*bulk_num+1} to {ind*bulk_num+bulk_num}")
        field_names = df_target_entity.loc[ind*bulk_num+1:ind*bulk_num+bulk_num,'field_name']
        index = field_names.index
        field_lst = [data.find_field(x) for x in list(field_names)]

        df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
        df_download = df_download.toPandas()

        df_download.to_csv(f'data/{entity}/{ind}.csv',index=False)

        df_target_entity.loc[index, 'downloaded']=[True]*len(index)
        df_target_entity.loc[index, 'ind']=[ind]*len(index)
        df_target_entity.to_csv(f'recorder/recorder_{entity}.csv')
    except:
        print(f"error in bulk {ind}")
        fail_list+=[ind]
    ind+=1
print(f' starttime {start_time}, endtime {datetime.now()}')

now download bulk 18, from 541 to 570
error in bulk 18
now download bulk 19, from 571 to 600
error in bulk 19
now download bulk 20, from 601 to 630
error in bulk 20
now download bulk 21, from 631 to 660
error in bulk 21
now download bulk 22, from 661 to 690
error in bulk 22
now download bulk 23, from 691 to 720
error in bulk 23
now download bulk 24, from 721 to 750
error in bulk 24
now download bulk 25, from 751 to 780
error in bulk 25
now download bulk 26, from 781 to 810
error in bulk 26
now download bulk 27, from 811 to 840
error in bulk 27
now download bulk 28, from 841 to 870
error in bulk 28
now download bulk 29, from 871 to 900
error in bulk 29
now download bulk 30, from 901 to 930
error in bulk 30
now download bulk 31, from 931 to 960
error in bulk 31
now download bulk 32, from 961 to 990
error in bulk 32
now download bulk 33, from 991 to 1020
error in bulk 33
now download bulk 34, from 1021 to 1050
error in bulk 34
now download bulk 35, from 1051 to 1080
error in bulk 35
now d

In [25]:
list(range(18,32))

[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

In [21]:
from datetime import datetime
start_time = datetime.now()
df_target_entity = pd.read_csv(f'recorder/recorder_{entity}.csv')
fail_list = [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,52, 68, 179, 815, 816, 820, 828]

for ind in fail_list:
    try:
        print(f"now download bulk {ind}, from {ind*bulk_num+1} to {ind*bulk_num+bulk_num}")
        field_names = df_target_entity.loc[ind*bulk_num+1:ind*bulk_num+bulk_num,'field_name']
        index = field_names.index
        field_lst = [data.find_field(x) for x in list(field_names)]

        df_download = data.retrieve_fields(engine=engine, fields=field_lst, coding_values="replace")
        df_download = df_download.toPandas()

        df_download.to_csv(f'data/{entity}/{ind}.csv',index=False)

        df_target_entity.loc[index, 'downloaded']=[True]*len(index)
        df_target_entity.loc[index, 'ind']=[ind]*len(index)
        df_target_entity.to_csv(f'recorder/recorder_{entity}.csv')
    except Exception as e:
        print(e)
        print(f"error in bulk {ind}")
        fail_list+=[ind]
    ind+=1
print(f' starttime {start_time}, endtime {datetime.now()}')

now download bulk 17, from 511 to 540
[Errno 111] Connection refused
error in bulk 17
now download bulk 18, from 541 to 570
[Errno 111] Connection refused
error in bulk 18
now download bulk 52, from 1561 to 1590


KeyboardInterrupt: 

ind = 0
interval = 5

filenames = [f'data/{entity}/{x}.csv' for x in range(ind,ind+interval)]
df_compact = pd.concat(map(pd.read_csv, filenames))
df_compact.to_pickle(f'data/{entity}/{ind}-{interval-1}.pkl')
for file in filenames:
    os.remove(file)
